# GRU Classification with MR Dataset
<hr>

We will build a text classification model using GRU model on the MR Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [3]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [4]:
corpus = pd.read_pickle('../../../0_data/MR/MR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10662, 3)


,sentence,label,split
0,"simplistic , silly and tedious .",0,train
1,"it 's so laddish and juvenile , only teenage b...",0,train
2,exploitative and largely devoid of the depth o...,0,train
3,garbus discards the potential for pathological...,0,train
4,a visually flashy but narratively opaque and e...,0,train
...,...,...,...
10657,both exuberantly romantic and serenely melanch...,1,train
10658,mazel tov to a film about a family 's joyous l...,1,train
10659,standing in the shadows of motown is the best ...,1,train
10660,it 's nice to see piscopo again after all thes...,1,train


In [5]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10662 entries, 0 to 10661
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10662 non-null  object
 1   label     10662 non-null  int32 
 2   split     10662 non-null  object
dtypes: int32(1), object(2)
memory usage: 208.4+ KB


In [6]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,5331,5331
1,5331,5331


In [7]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [8]:
sentences[0]

'simplistic , silly and tedious .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [9]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification .
Into a sequence of int: [3, 544, 1838, 13, 3909, 3366, 4, 658, 2629, 416, 10, 236, 4, 10112]
Into a padded sequence: [    3   544  1838    13  3909  3366     4   658  2629   416    10   236
     4 10112     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


In [11]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
is 7
's 8
it 9
in 10
18760


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## GRU Model

In [12]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [13]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          140544    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               74496     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 515,169
Trainable params: 515,169
Non-trainable params: 0
_________________________________________________________________


In [14]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [15]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=30, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/30
300/300 [==============================] - 91s 207ms/step - loss: 0.6162 - accuracy: 0.6261 - val_loss: 0.4727 - val_accuracy: 0.7723
Epoch 2/30
300/300 [==============================] - 59s 197ms/step - loss: 0.2163 - accuracy: 0.9175 - val_loss: 0.5376 - val_accuracy: 0.7638
Epoch 3/30
300/300 [==============================] - 54s 180ms/step - loss: 0.0619 - accuracy: 0.9828 - val_loss: 0.8385 - val_accuracy: 0.7479
Epoch 4/30
300/300 [==============================] - 51s 171ms/step - loss: 0.0210 - accuracy: 0.9944 - val_loss: 1.1805 - val_accuracy: 0.7582
Epoch 5/30
300/300 [==============================] - 52s 174ms/step - loss: 0.0080 - accuracy: 0.9981 - val_loss: 1.3872 - val_accuracy: 0.7423
Epoch 6/30
300/300 [==============================] - 55s 183ms/step - loss: 0.0033 - accuracy: 0.9989 - val_loss: 1.6922 - val_accuracy: 0.7498
Epoch 7/30
300/300 [==============================] - 62s 206ms/step - loss: 0.0161 - accuracy: 0.9956 - val_loss: 1.3107 - val_ac

Epoch 10/30
300/300 [==============================] - 50s 167ms/step - loss: 0.0046 - accuracy: 0.9988 - val_loss: 1.7120 - val_accuracy: 0.7355
Epoch 11/30
300/300 [==============================] - 56s 187ms/step - loss: 0.0098 - accuracy: 0.9962 - val_loss: 1.5964 - val_accuracy: 0.7402
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 77.11069583892822
Training 6: 
Epoch 1/30
300/300 [==============================] - 88s 202ms/step - loss: 0.6195 - accuracy: 0.6151 - val_loss: 0.4792 - val_accuracy: 0.7533
Epoch 2/30
300/300 [==============================] - 50s 165ms/step - loss: 0.2263 - accuracy: 0.9129 - val_loss: 0.5940 - val_accuracy: 0.7392
Epoch 3/30
300/300 [==============================] - 49s 164ms/step - loss: 0.0647 - accuracy: 0.9787 - val_loss: 0.9533 - val_accuracy: 0.7280
Epoch 4/30
300/300 [==============================] - 49s 164ms/step - loss: 0.0228 - accuracy: 0.9934 - val_loss: 1.0524 - val_accuracy: 0.724

Epoch 5/30
300/300 [==============================] - 56s 188ms/step - loss: 0.0082 - accuracy: 0.9961 - val_loss: 1.2836 - val_accuracy: 0.7598
Epoch 6/30
300/300 [==============================] - 54s 179ms/step - loss: 0.0047 - accuracy: 0.9981 - val_loss: 1.5718 - val_accuracy: 0.7749
Epoch 7/30
300/300 [==============================] - 53s 177ms/step - loss: 0.0048 - accuracy: 0.9988 - val_loss: 1.2794 - val_accuracy: 0.7580
Epoch 8/30
300/300 [==============================] - 52s 174ms/step - loss: 0.0116 - accuracy: 0.9953 - val_loss: 1.1543 - val_accuracy: 0.7627
Epoch 9/30
300/300 [==============================] - 52s 173ms/step - loss: 0.0097 - accuracy: 0.9971 - val_loss: 1.6490 - val_accuracy: 0.7345
Epoch 10/30
300/300 [==============================] - 52s 173ms/step - loss: 0.0046 - accuracy: 0.9980 - val_loss: 1.6621 - val_accuracy: 0.7542
Epoch 11/30
300/300 [==============================] - 52s 172ms/step - loss: 0.0094 - accuracy: 0.9968 - val_loss: 1.3954 - val_

## Summary

In [16]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,77.22587,77.413309,77.204502,75.046903,77.110696,75.328332,77.016884,76.641649,78.61163,77.485931,76.908571


In [17]:
report = record
report = report.to_excel('GRU_MR_v2.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [18]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [19]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [20]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [21]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 16448 words present from 18760 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [22]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [23]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.25246422,  1.40718628, -0.3619654 , ..., -0.80374242,
        -1.59198165, -0.68927201],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## GRU Model

In [24]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [25]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 100, 128)          140544    
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 128)               74496     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 515,169
Trainable params: 215,169
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [26]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [27]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/40
300/300 [==============================] - 72s 150ms/step - loss: 0.6255 - accuracy: 0.6263 - val_loss: 0.5543 - val_accuracy: 0.7207
Epoch 2/40
300/300 [==============================] - 31s 102ms/step - loss: 0.4692 - accuracy: 0.7764 - val_loss: 0.6017 - val_accuracy: 0.7170
Epoch 3/40
300/300 [==============================] - 31s 102ms/step - loss: 0.4252 - accuracy: 0.8029 - val_loss: 0.6452 - val_accuracy: 0.7170
Epoch 4/40
300/300 [==============================] - 31s 104ms/step - loss: 0.3859 - accuracy: 0.8239 - val_loss: 0.6506 - val_accuracy: 0.7263
Epoch 5/40
300/300 [==============================] - 31s 103ms/step - loss: 0.3326 - accuracy: 0.8480 - val_loss: 0.5954 - val_accuracy: 0.7470
Epoch 6/40
300/300 [==============================] - 31s 105ms/step - loss: 0.2834 - accuracy: 0.8830 - val_loss: 0.5916 - val_accuracy: 0.7488
Epoch 7/40
300/300 [==============================] - 38s 126ms/step - loss: 0.2416 - accuracy: 0.9053 - val_loss: 0.6936 - val_ac

Test Accuracy: 78.23639512062073
Training 4: 
Epoch 1/40
300/300 [==============================] - 78s 129ms/step - loss: 0.6266 - accuracy: 0.6380 - val_loss: 0.5167 - val_accuracy: 0.7411
Epoch 2/40
300/300 [==============================] - 31s 103ms/step - loss: 0.4575 - accuracy: 0.7840 - val_loss: 0.4944 - val_accuracy: 0.7514
Epoch 3/40
300/300 [==============================] - 31s 103ms/step - loss: 0.3946 - accuracy: 0.8241 - val_loss: 0.5092 - val_accuracy: 0.7467
Epoch 4/40
300/300 [==============================] - 31s 103ms/step - loss: 0.3585 - accuracy: 0.8417 - val_loss: 0.5298 - val_accuracy: 0.7580
Epoch 5/40
300/300 [==============================] - 31s 104ms/step - loss: 0.3220 - accuracy: 0.8631 - val_loss: 0.5996 - val_accuracy: 0.7383
Epoch 6/40
300/300 [==============================] - 31s 104ms/step - loss: 0.2714 - accuracy: 0.8875 - val_loss: 0.6242 - val_accuracy: 0.7402
Epoch 7/40
300/300 [==============================] - 31s 103ms/step - loss: 0.2310 

300/300 [==============================] - 39s 131ms/step - loss: 0.0280 - accuracy: 0.9909 - val_loss: 2.0058 - val_accuracy: 0.7205
Epoch 19/40
300/300 [==============================] - 33s 110ms/step - loss: 0.0178 - accuracy: 0.9948 - val_loss: 2.0512 - val_accuracy: 0.7101
Epoch 20/40
300/300 [==============================] - 33s 111ms/step - loss: 0.0126 - accuracy: 0.9956 - val_loss: 2.1287 - val_accuracy: 0.7317
Epoch 21/40
300/300 [==============================] - 33s 111ms/step - loss: 0.0133 - accuracy: 0.9967 - val_loss: 2.5557 - val_accuracy: 0.7026
Epoch 22/40
300/300 [==============================] - 36s 120ms/step - loss: 0.0143 - accuracy: 0.9958 - val_loss: 1.9884 - val_accuracy: 0.7467
Epoch 23/40
300/300 [==============================] - 35s 117ms/step - loss: 0.0180 - accuracy: 0.9937 - val_loss: 2.1020 - val_accuracy: 0.7139
Epoch 24/40
300/300 [==============================] - 30s 101ms/step - loss: 0.0272 - accuracy: 0.9911 - val_loss: 1.9449 - val_accurac

300/300 [==============================] - 31s 103ms/step - loss: 0.0931 - accuracy: 0.9653 - val_loss: 0.8037 - val_accuracy: 0.7908
Epoch 12/40
300/300 [==============================] - 31s 102ms/step - loss: 0.0597 - accuracy: 0.9791 - val_loss: 0.9640 - val_accuracy: 0.7992
Epoch 13/40
300/300 [==============================] - 31s 103ms/step - loss: 0.0442 - accuracy: 0.9834 - val_loss: 0.9351 - val_accuracy: 0.7936
Epoch 14/40
300/300 [==============================] - 31s 103ms/step - loss: 0.0410 - accuracy: 0.9857 - val_loss: 1.0162 - val_accuracy: 0.7927
Epoch 15/40
300/300 [==============================] - 31s 102ms/step - loss: 0.0237 - accuracy: 0.9922 - val_loss: 1.0524 - val_accuracy: 0.7861
Epoch 16/40
300/300 [==============================] - 31s 102ms/step - loss: 0.0350 - accuracy: 0.9868 - val_loss: 1.1648 - val_accuracy: 0.7805
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 80.7692289352417
Training 10: 
Epoch 

## Summary

In [28]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,76.944703,75.25773,78.236395,75.797373,75.046903,74.671668,78.330207,76.923078,80.769229,79.455912,77.14332


In [29]:
report = record2
report = report.to_excel('GRU_MR_v2_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## GRU Model

In [30]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [31]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_44 (Bidirectio (None, 100, 128)          140544    
_________________________________________________________________
bidirectional_45 (Bidirectio (None, 128)               74496     
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 129       
Total params: 515,169
Trainable params: 515,169
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [32]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [33]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record3)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/40
300/300 [==============================] - 85s 199ms/step - loss: 0.6130 - accuracy: 0.6383 - val_loss: 0.4564 - val_accuracy: 0.7713
Epoch 2/40
300/300 [==============================] - 51s 171ms/step - loss: 0.2777 - accuracy: 0.8833 - val_loss: 0.5160 - val_accuracy: 0.7591
Epoch 3/40
300/300 [==============================] - 52s 174ms/step - loss: 0.1076 - accuracy: 0.9624 - val_loss: 0.6985 - val_accuracy: 0.7676
Epoch 4/40
300/300 [==============================] - 50s 167ms/step - loss: 0.0374 - accuracy: 0.9880 - val_loss: 1.0300 - val_accuracy: 0.7413
Epoch 5/40
300/300 [==============================] - 48s 161ms/step - loss: 0.0118 - accuracy: 0.9975 - val_loss: 1.2321 - val_accuracy: 0.7591
Epoch 6/40
300/300 [==============================] - 50s 166ms/step - loss: 0.0041 - accuracy: 0.9987 - val_loss: 1.2953 - val_accuracy: 0.7498
Epoch 7/40
300/300 [==============================] - 51s 172ms/step - loss: 0.0120 - accuracy: 0.9952 - val_loss: 1.4578 - val_ac

Epoch 8/40
300/300 [==============================] - 55s 184ms/step - loss: 0.0071 - accuracy: 0.9982 - val_loss: 1.4257 - val_accuracy: 0.7814
Epoch 9/40
300/300 [==============================] - 60s 200ms/step - loss: 0.0074 - accuracy: 0.9980 - val_loss: 1.2272 - val_accuracy: 0.7861
Epoch 10/40
300/300 [==============================] - 55s 184ms/step - loss: 0.0053 - accuracy: 0.9986 - val_loss: 1.3872 - val_accuracy: 0.7805
Epoch 11/40
300/300 [==============================] - 57s 191ms/step - loss: 0.0060 - accuracy: 0.9983 - val_loss: 1.4827 - val_accuracy: 0.7880
Epoch 12/40
300/300 [==============================] - 60s 199ms/step - loss: 9.7030e-04 - accuracy: 0.9998 - val_loss: 1.3733 - val_accuracy: 0.7861
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 80.86304068565369
Training 6: 
Epoch 1/40
300/300 [==============================] - 89s 210ms/step - loss: 0.6050 - accuracy: 0.6472 - val_loss: 0.5123 - val_accuracy: 

Epoch 1/40
300/300 [==============================] - 80s 190ms/step - loss: 0.6018 - accuracy: 0.6546 - val_loss: 0.4254 - val_accuracy: 0.8114
Epoch 2/40
300/300 [==============================] - 54s 179ms/step - loss: 0.2771 - accuracy: 0.8872 - val_loss: 0.4744 - val_accuracy: 0.7824
Epoch 3/40
300/300 [==============================] - 55s 182ms/step - loss: 0.1063 - accuracy: 0.9616 - val_loss: 0.6077 - val_accuracy: 0.7767
Epoch 4/40
300/300 [==============================] - 53s 176ms/step - loss: 0.0389 - accuracy: 0.9890 - val_loss: 0.8475 - val_accuracy: 0.7786
Epoch 5/40
300/300 [==============================] - 54s 181ms/step - loss: 0.0151 - accuracy: 0.9961 - val_loss: 1.1102 - val_accuracy: 0.7608
Epoch 6/40
300/300 [==============================] - 52s 173ms/step - loss: 0.0089 - accuracy: 0.9976 - val_loss: 1.2496 - val_accuracy: 0.7730
Epoch 7/40
300/300 [==============================] - 52s 173ms/step - loss: 0.0042 - accuracy: 0.9977 - val_loss: 1.2765 - val_ac

## Summary

In [34]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,77.132148,79.381442,78.705442,78.330207,80.863041,76.360226,76.266414,78.705442,77.485931,81.144464,78.437476


In [35]:
report = record3
report = report.to_excel('GRU_MR_v2_3.xlsx', sheet_name='dynamic')